<a href="https://colab.research.google.com/github/TikaAbstract/Spring-E-commerce-project/blob/main/assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip show gensim


Name: gensim
Version: 4.3.2
Summary: Python framework for fast Vector Space Modelling
Home-page: https://radimrehurek.com/gensim/
Author: Radim Rehurek
Author-email: me@radimrehurek.com
License: LGPL-2.1-only
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, scipy, smart-open
Required-by: 


In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors
import csv
import pandas as pd

# Load the pre-trained 'word2vec-google-news-300' model
model = api.load("word2vec-google-news-300")

# Read the synonym.csv file and create other 2 files
dataset = pd.read_csv('/content/synonym.csv')
details_filename = 'word2vec-google-news-300-details.csv'
analysis_filename = 'analysis.csv'

# Variables to calculate during the analysis
total_correct = 0
total_answered = 0
vocabulary_size = len(model.key_to_index)

# Evaluate the dataset and write the details to the (model)details.csv
with open(details_filename, mode='w', newline='') as details_file:
    details_writer = csv.writer(details_file)

    for index, row in dataset.iterrows():
        question_word = row['question']
        correct_answer = row['answer']
        options = [row[str(i)] for i in range(4)]

        if question_word in model.key_to_index:
            highest_similarity = -1
            best_guess = None

            for option in options:
                if option in model.key_to_index:
                    similarity = model.similarity(question_word, option)
                    if similarity > highest_similarity:
                        highest_similarity = similarity
                        best_guess = option

            if best_guess:
                label = 'correct' if best_guess == correct_answer else 'wrong'
                total_answered += 1
                total_correct += 1 if label == 'correct' else 0
            else:
                label = 'guess'

            details_writer.writerow([question_word, correct_answer, best_guess if best_guess else 'N/A', label])
        else:
            details_writer.writerow([question_word, correct_answer, 'N/A', 'guess'])

# Calculate the accuracy
accuracy = total_correct / total_answered if total_answered > 0 else 0

# Write the analysis.csv file
with open(analysis_filename, mode='w', newline='') as analysis_file:
    analysis_writer = csv.writer(analysis_file)
    analysis_writer.writerow(['word2vec-google-news-300', vocabulary_size, total_correct, total_answered, accuracy])

# Creating the analysis result to be outputed
analysis_result = {
    'model_name': 'word2vec-google-news-300',
    'vocabulary_size': vocabulary_size,
    'correct_labels': total_correct,
    'total_answered_without_guessing': total_answered,
    'accuracy': accuracy
}

# Output the analysis result to the user
print(analysis_result)


[==================================================] 100.0% 1662.8/1662.8MB downloaded
{'model_name': 'word2vec-google-news-300', 'vocabulary_size': 3000000, 'correct_labels': 70, 'total_answered_without_guessing': 79, 'accuracy': 0.8860759493670886}


In [ ]:
# task_2.3
import gensim.downloader as api
from gensim.models import KeyedVectors
import csv
import pandas as pd

def evaluate_model(model_name, dataset_path, details_filename, analysis_filename):
    # Load the pre-trained model
    model = api.load(model_name)

    # Read the dataset
    dataset = pd.read_csv(dataset_path)

    # Variables to calculate during the analysis
    total_correct = 0
    total_answered = 0
    vocabulary_size = len(model.key_to_index)

    # Evaluate the dataset and write the details
    with open(details_filename, mode='w', newline='') as details_file:
        details_writer = csv.writer(details_file)

        for _, row in dataset.iterrows():
            question_word = row['question']
            correct_answer = row['answer']
            options = [row[str(i)] for i in range(4)]

            if question_word in model.key_to_index:
                highest_similarity = -1
                best_guess = None

                for option in options:
                    if option in model.key_to_index:
                        similarity = model.similarity(question_word, option)
                        if similarity > highest_similarity:
                            highest_similarity = similarity
                            best_guess = option

                label = 'correct' if best_guess == correct_answer else 'wrong'
                total_answered += 1
                total_correct += 1 if label == 'correct' else 0

            else:
                label = 'guess'

            details_writer.writerow([question_word, correct_answer, best_guess if best_guess else 'N/A', label])

    # Calculate the accuracy
    accuracy = total_correct / total_answered if total_answered > 0 else 0

    # Append to the analysis.csv file
    with open(details_filename, mode='w', newline='') as details_file:
      details_writer = csv.writer(details_file)

    for _, row in dataset.iterrows():
        question_word = row['question']
        correct_answer = row['answer']
        options = [row[str(i)] for i in range(4)]  # Adjusted to use correct column names

# Define your dataset path
dataset_path = '/content/synonym.csv'  # Replace with your actual file path

# Evaluate for the first model
evaluate_model('word2vec-google-news-300', dataset_path, 'word2vec-google-news-300-details.csv', 'analysis.csv')

# Evaluate for other models
evaluate_model('glove-twitter-100', dataset_path, 'glove-twitter-100-details.csv', 'analysis.csv')
evaluate_model('glove-wiki-gigaword-100', dataset_path, 'glove-wiki-gigaword-100-details.csv', 'analysis.csv')

evaluate_model('glove-wiki-gigaword-50', dataset_path, 'glove-wiki-gigaword-50-details.csv', 'analysis.csv')
evaluate_model('glove-wiki-gigaword-300', dataset_path, 'glove-wiki-gigaword-300-details.csv', 'analysis.csv')

# Append to the analysis.csv file in append mode
with open(analysis_filename, mode='a', newline='') as analysis_file:
    analysis_writer = csv.writer(analysis_file)
    analysis_writer.writerow([model_name, vocabulary_size, total_correct, total_answered, accuracy])



[==================================================] 100.0% 387.1/387.1MB downloaded
[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
# 2.3 Task2:Comparison with Other Pre-trained Models

import gensim.downloader as api
from gensim.models import KeyedVectors
import csv
import pandas as pd

# Function to evaluate a model
def evaluate_model(model_name, dataset_path, details_filename, analysis_filename):
    # Load the model
    model = api.load(model_name)

    # Read the dataset
    dataset = pd.read_csv(dataset_path)

    # Initialize variables for analysis
    total_correct = 0
    total_answered = 0
    vocabulary_size = len(model.key_to_index)

    # Evaluate the dataset and write the details
    with open(details_filename, mode='w', newline='') as details_file:
        details_writer = csv.writer(details_file)

        for _, row in dataset.iterrows():
            question_word = row['question']
            correct_answer = row['answer']
            options = [row[str(i)] for i in range(4)]

            if question_word in model.key_to_index:
                highest_similarity = -1
                best_guess = None

                for option in options:
                    if option in model.key_to_index:
                        similarity = model.similarity(question_word, option)
                        if similarity > highest_similarity:
                            highest_similarity = similarity
                            best_guess = option

                label = 'correct' if best_guess == correct_answer else 'wrong'
                total_answered += 1
                total_correct += 1 if label == 'correct' else 0
            else:
                label = 'guess'

            details_writer.writerow([question_word, correct_answer, best_guess if best_guess else 'N/A', label])

    # Calculate the accuracy
    accuracy = total_correct / total_answered if total_answered > 0 else 0

    # Append to the analysis.csv file
    with open(analysis_filename, mode='a', newline='') as analysis_file:
        analysis_writer = csv.writer(analysis_file)
        analysis_writer.writerow([model_name, vocabulary_size, total_correct, total_answered, accuracy])

# Define your dataset path
dataset_path = '/content/synonym.csv'

# Evaluate for selected models
evaluate_model('glove-twitter-100', dataset_path, 'glove-twitter-100-details.csv', 'analysis.csv')
evaluate_model('glove-wiki-gigaword-100', dataset_path, 'glove-wiki-gigaword-100-details.csv', 'analysis.csv')

evaluate_model('glove-wiki-gigaword-50', dataset_path, 'glove-wiki-gigaword-50-details.csv', 'analysis.csv')
evaluate_model('glove-wiki-gigaword-300', dataset_path, 'glove-wiki-gigaword-300-details.csv', 'analysis.csv')


[==================================================] 100.0% 66.0/66.0MB downloaded
[==================================================] 100.0% 376.1/376.1MB downloaded
